In [1]:
from jinja2 import Template
import re
import string
import snowflake.connector
import pandas as pd

/Users/andrea.salvati/.pyenv/versions/3.10.0/envs/ads-fixtures-lookup/lib/python3.10/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (8.0.0), please install a version that adheres to: 'pyarrow<6.1.0,>=6.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
# define snowflake connection
ctx = snowflake.connector.connect(
    user='TRANSFORMATION_DEV_SERVICE',
    password='T4aN5_D3w_S3erV#',
    account='si44367.eu-central-1',
    #schema='STAGING',
    #database='TRANSFORMATION_DEV',
    warehouse='TRANSFORMATION_DEV_S_WH'
    )

# define run query function 
def run_query(query):
    cs=ctx.cursor()
    allrows=cs.execute(query).fetchall()
    # write the results of the query into a pandas dataframe 
    df = pd.DataFrame(allrows, columns=[i[0] for i in cs.description])
    # return the dataframe
    return df

# Test for application_category_parser

In [9]:
# define function to test
func_name = 'application_category_parser'
test_value = 'skyq'

In [103]:
# read the function
with open('functions' + "/" + func_name + ".sql", 'r') as jinja_file:
    jinja_func = jinja_file.read()

In [105]:
# cast the function to string and clean it
clean_function = jinja_func.split('\n')[2:][:-2]
str_func = ' '.join(clean_function)
print(str_func)

CASE             WHEN LOWER("{{ application_type }}") IN ('smartcast'                                                 ,'has_player'                                                 ,'tizen'                                                 ,'xfinity'                                                 ,'fairplay'                                                 ,'webos'                                                 ,'exoplayer'                                                 ,'abstract method'                                                 ,'tv web'                                                 ,'skyq'                                                 ,'mdk_sky_q'                                                 ,'roku tv'                                                 ,'shaka'                                                 ,'netcast'                                                 ,'orsay'                                                 ,'opera tv'                                         

In [114]:
jinja_func_temp = Template(str_func)
jinja_func_final = jinja_func_temp.render({"application_type": 'skyq'})
jinja_func_final = jinja_func_final.replace('"',"'")

In [118]:
run_query('SELECT ' + jinja_func_final)

,output_value
0,Living Room


# test for cdn_parser

In [119]:
# define function to test
func_name = 'cdn_parser'

In [120]:
# read the function
with open('functions' + "/" + func_name + ".sql", 'r') as jinja_file:
    jinja_func = jinja_file.read()

In [121]:
# cast the function to string and clean it
clean_function = jinja_func.split('\n')[2:][:-2]
str_func = ' '.join(clean_function)
print(str_func)

CASE     WHEN "{{ start_time_unix }}" >= 1506124800 AND LEFT("{{ stream_url }}",40) ILIKE '%akamai%' THEN 'Akamai'     WHEN "{{ start_time_unix }}" >= 1506124800 AND LEFT("{{ stream_url }}",40) ILIKE '%ll%' THEN 'Limelight'     WHEN "{{ start_time_unix }}" >= 1506124800 AND LEFT("{{ stream_url }}",40) ILIKE '%cf%' THEN 'Amazon'     WHEN "{{ start_time_unix }}" >= 1506124800 AND "{{ stream_url }}" ILIKE '%perform.secure.footprint.net%' THEN 'CenturyLink'     WHEN "{{ start_time_unix }}" >= 1506124800 AND LEFT("{{ stream_url }}",40) ILIKE '%fw%' THEN 'Flow'     WHEN "{{ start_time_unix }}" >= 1506124800 AND LEFT("{{ stream_url }}",40) ILIKE '%vn%' THEN 'Verizon'     WHEN "{{ start_time_unix }}" >= 1506124800 AND LEFT("{{ stream_url }}",40) ILIKE '%un%' THEN 'UDN'     WHEN "{{ start_time_unix }}" >= 1506124800 AND LEFT("{{ stream_url }}",40) ILIKE '%dr%' THEN 'DLVR'     WHEN "{{ start_time_unix }}" >= 1506124800 AND LEFT("{{ stream_url }}",40) ILIKE '%ac%' THEN 'Amazon'     WHEN "{{ start

In [146]:
# isolate the arguments of the function
args = re.findall(r"{{ (.*?) }}", str_func)
args = list(dict.fromkeys(args))
args


['start_time_unix', 'stream_url']

In [131]:
jinja_func_temp = Template(str_func)

In [21]:
access_history = run_query("""
SELECT 
    BASE_OBJECT_NAME
    ,MODIFIED_OBJECT_NAME
    ,MIN(QUERY_START_TIME) AS MIN_QUERY_START_TIME
FROM "TRANSFORMATION_PROD"."METADATA"."MM_ACCESS_HISTORY" 
WHERE 1=1
    AND "QUERY_START_TIME" > DATEADD('day', -1, CURRENT_DATE) 
    AND MODIFIED_OBJECT_NAME IS NOT NULL
    AND BASE_OBJECT_NAME <> MODIFIED_OBJECT_NAME
    AND BASE_OBJECT_NAME IS NOT NULL
    AND BASE_DATABASE_NAME IN ('PRD_PROD', 'EDM_PROD')
    AND MODIFIED_DATABASE_NAME IN ('PRD_PROD', 'EDM_PROD')
GROUP BY 1,2
ORDER BY MIN_QUERY_START_TIME DESC
""")

In [22]:
access_history

,BASE_OBJECT_NAME,MODIFIED_OBJECT_NAME,MIN_QUERY_START_TIME
0,"""PRD_PROD.ANALYTICS.\""tziei\""""","""PRD_PROD.ANALYTICS.\""custom_trigger_logs\""""",2022-05-30 08:56:32.448000+00:00
1,"""PRD_PROD.STAGING.\""opreng_JSE_SUB_BASE_JOINED...","""PRD_PROD.DMT_LOOKER.\""operator_engagement_mon...",2022-05-30 08:54:34.775000+00:00
2,"""PRD_PROD.STAGING.\""opreng_TEST5_MONTHLY\""""","""PRD_PROD.STAGING.\""opreng_JSE_SUB_BASE_JOINED...",2022-05-30 08:54:04.367000+00:00
3,"""PRD_PROD.STAGING.\""opreng_JSE_SUB_BASE_JOINED...","""PRD_PROD.STAGING.\""opreng_JSE_SUB_BASE_JOINED...",2022-05-30 08:54:04.367000+00:00
4,"""PRD_PROD.STAGING.\""opreng_ENGAGED_MONTHLY_L1\""""","""PRD_PROD.STAGING.\""opreng_JSE_SUB_BASE_JOINED...",2022-05-30 08:54:04.367000+00:00
...,...,...,...
4157,"""EDM_PROD.OPEN_DOTS.\""Messages\""""","""PRD_PROD.STAGING.\""whtonv_VODPRODUCT_LATEST\""""",2022-05-29 00:03:19.252000+00:00
4158,"""EDM_PROD.OPEN_DOTS.\""Messages\""""","""PRD_PROD.STAGING.\""whtonr_CONTRACT_LATEST\""""",2022-05-29 00:03:16.115000+00:00
4159,"""EDM_PROD.OPEN_DOTS.\""Messages\""""","""PRD_PROD.STAGING.\""whtonr_RIGHTSHOLDER_LATEST\""""",2022-05-29 00:03:15.789000+00:00
4160,"""EDM_PROD.OPEN_DOTS.\""Messages\""""","""PRD_PROD.STAGING.\""whtonr_LIVERIGHT_LATEST\""""",2022-05-29 00:03:15.474000+00:00


In [10]:
import sys
sys.setrecursionlimit(20000)

In [12]:
table_name = '"PRD_PROD.ANALYTICS.\\"watch_party_sessions_enriched\\""'
lineage = [table_name]


def run_loop(table_name,df):
    lineage = [table_name]
    for index,table in enumerate(df['MODIFIED_OBJECT_NAME']):
        #print(index,table)
        if table == table_name:
            if len(df[df['MODIFIED_OBJECT_NAME']==table_name]) == 1 :
            #print(table)
                lineage.append(df['BASE_OBJECT_NAME'].iloc[index])
                table_name = df['BASE_OBJECT_NAME'].iloc[index] 
                #df_1=df.drop(index)
                #print(df_1)
                run_loop(table_name,df)      
               # print(lineage)
            else:
                lsite=df[df['MODIFIED_OBJECT_NAME']==table_name]['BASE_OBJECT_NAME'].to_list()
                #print(df[df['MODIFIED_OBJECT_NAME']==table_name]['BASE_OBJECT_NAME'].to_list())
                for i in lsite:
                    #print(i)
                    lineage.append(i)
                    table_name = i
                    run_loop(table_name,df)
    print(lineage)

In [ ]:
lineage_fun = run_loop('"PRD_PROD.ANALYTICS.\\"watch_party_sessions_enriched\\""', access_history)

In [11]:
lineage

['"PRD_PROD.ANALYTICS.\\"watch_party_sessions_enriched\\""']

In [ ]:
table_name = ['"PRD_PROD.ANALYTICS.\\"watch_party_sessions_enriched\\""']
last_input_table = ''
lineage = [table_name]
i=0

while i <= len(access_history['BASE_OBJECT_NAME']):
    for input_table in table_name:
        #print(input_table)
        for index,table in enumerate(access_history['MODIFIED_OBJECT_NAME']):
            if (table == input_table):
                lineage.append(access_history['BASE_OBJECT_NAME'].iloc[index])
                table_name = access_history[access_history['MODIFIED_OBJECT_NAME'] == input_table]['BASE_OBJECT_NAME']
                #print(table_name)
    i += 1
    print(i)

In [33]:
lineage_new = list(dict.fromkeys(lineage[1:]))
lineage_new

['"PRD_PROD.ANALYTICS.\\"watch_party_sessions_enriched__dbt_tmp\\""',
 '"PRD_PROD.ANALYTICS.\\"watch_party_sessions_enriched\\""',
 '"PRD_PROD.STAGING.\\"watch_party_load\\""']

In [36]:
table_name = '"PRD_PROD.ANALYTICS.\\"watch_party_sessions_enriched\\""'
lineage = [table_name]

def get_lineage(table_name,df):
    tables_used=df[df['MODIFIED_OBJECT_NAME']==table_name]['BASE_OBJECT_NAME'].to_list()
    for i in tables_used:
        if i in lineage:
            continue
        else:
            lineage.append(i)
            print(i)
            get_lineage(i,df)
    return(lineage)


In [38]:
get_lineage(table_name,access_history)

['"PRD_PROD.ANALYTICS.\\"watch_party_sessions_enriched\\""',
 '"PRD_PROD.ANALYTICS.\\"watch_party_sessions_enriched__dbt_tmp\\""',
 '"PRD_PROD.STAGING.\\"watch_party_load\\""',
 '"PRD_PROD.STAGING.\\"watch_party_daily_load\\""',
 '"PRD_PROD.STAGING.PLAYS_FACT_SAMPLE"',
 '"PRD_PROD.ANALYTICS.\\"plays_fact_dn\\""',
 '"PRD_PROD.ANALYTICS.\\"plays_fact_dn__dbt_tmp\\""',
 '"PRD_PROD.STAGING.\\"join_errors_back_in\\""',
 '"PRD_PROD.STAGING.\\"plays_fact_dn_temp\\""',
 '"PRD_PROD.STAGING.\\"b2c_pfdn_pre1\\""',
 '"PRD_PROD.ANALYTICS.\\"region_dimension\\""',
 '"PRD_PROD.STAGING.UNION_CUSTOMER_ROWS"',
 '"PRD_PROD.STAGING.GET_ALL_ROWS"',
 '"PRD_PROD.ANALYTICS.\\"customer_history_scd\\""',
 '"PRD_PROD.STAGING.\\"non_quarantined_scd_data\\""',
 '"PRD_PROD.STAGING.\\"sql_customer_dimension_quarantine_dt\\""',
 '"PRD_PROD.STAGING.\\"partition_customer_scd_temp_dt\\""',
 '"PRD_PROD.STAGING.\\"join_add_detail_temp_dt\\""',
 '"PRD_PROD.ANALYTICS.\\"oem_partners_dimension\\""',
 '"PRD_PROD.STAGING.\\"pa

In [39]:
lineage[-30:]

['"PRD_PROD.STAGING.\\"get_livestream\\""',
 '"PRD_PROD.STAGING.\\"get_sports\\""',
 '"PRD_PROD.STAGING.\\"get_articles\\""',
 '"PRD_PROD.STAGING.\\"get_competitions\\""',
 '"PRD_PROD.STAGING.\\"join_up_mta_articles\\""',
 '"PRD_PROD.STAGING.\\"get_mta_articles\\""',
 '"PRD_PROD.STAGING.\\"backfill_content_dimension\\""',
 '"EDM_PROD.\\"Redshift_V1\\".CONTENT_DIMENSION"',
 '"PRD_PROD.ANALYTICS.\\"won_live_event_enriched\\""',
 '"PRD_PROD.STAGING.\\"whtonr_WON_LIVE_EVENT_ENRICHED_STAGING\\""',
 '"PRD_PROD.STAGING.\\"whtonr_CONTRACT_LATEST\\""',
 '"PRD_PROD.STAGING.\\"whtonr_REGION_LATEST\\""',
 '"PRD_PROD.STAGING.\\"whtonr_LIVEEVENT_FULL_SCHEDULE\\""',
 '"PRD_PROD.STAGING.\\"whtonr_LIVEEVENT_LATEST\\""',
 '"PRD_PROD.STAGING.\\"whtonr_PRELIMLIVEEVENT_LATEST\\""',
 '"PRD_PROD.STAGING.\\"whtonr_LIVEPRODUCT_LATEST\\""',
 '"PRD_PROD.STAGING.\\"whtonr_LIVERIGHT_LATEST\\""',
 '"PRD_PROD.STAGING.\\"whtonr_RIGHTSHOLDER_LATEST\\""',
 '"PRD_PROD.ANALYTICS.\\"device_dimension_adjusted\\""',
 '"PRD_

In [40]:
lineage_dedup = list(dict.fromkeys(lineage))
lineage_dedup

['"PRD_PROD.ANALYTICS.\\"watch_party_sessions_enriched\\""',
 '"PRD_PROD.ANALYTICS.\\"watch_party_sessions_enriched__dbt_tmp\\""',
 '"PRD_PROD.STAGING.\\"watch_party_load\\""',
 '"PRD_PROD.STAGING.\\"watch_party_daily_load\\""',
 '"PRD_PROD.STAGING.PLAYS_FACT_SAMPLE"',
 '"PRD_PROD.ANALYTICS.\\"plays_fact_dn\\""',
 '"PRD_PROD.ANALYTICS.\\"plays_fact_dn__dbt_tmp\\""',
 '"PRD_PROD.STAGING.\\"join_errors_back_in\\""',
 '"PRD_PROD.STAGING.\\"plays_fact_dn_temp\\""',
 '"PRD_PROD.STAGING.\\"b2c_pfdn_pre1\\""',
 '"PRD_PROD.ANALYTICS.\\"region_dimension\\""',
 '"PRD_PROD.STAGING.UNION_CUSTOMER_ROWS"',
 '"PRD_PROD.STAGING.GET_ALL_ROWS"',
 '"PRD_PROD.ANALYTICS.\\"customer_history_scd\\""',
 '"PRD_PROD.STAGING.\\"non_quarantined_scd_data\\""',
 '"PRD_PROD.STAGING.\\"sql_customer_dimension_quarantine_dt\\""',
 '"PRD_PROD.STAGING.\\"partition_customer_scd_temp_dt\\""',
 '"PRD_PROD.STAGING.\\"join_add_detail_temp_dt\\""',
 '"PRD_PROD.ANALYTICS.\\"oem_partners_dimension\\""',
 '"PRD_PROD.STAGING.\\"pa

In [54]:
table_name = 'test'
lineage = pd.DataFrame({"table_name": [table_name], "cost":[0]})

In [55]:
lineage

,table_name,cost
0,test,0


In [56]:
lineage['table_name']

0    test
Name: table_name, dtype: object

In [65]:
lineage = lineage.append({"table_name": table_name, "cost":0},ignore_index=True)

/var/folders/1f/n5xbqlfx0vnb8ql8thv5txg05tmqnf/T/ipykernel_15667/3300819255.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lineage = lineage.append({"table_name": table_name, "cost":0},ignore_index=True)


In [66]:
lineage

,table_name,cost
0,test,0
1,test,0
2,test,0
